In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
os.listdir("norm_data/")

['Plaquetas 03-2024.csv',
 'Hepatograma 03-2024.csv',
 'Glucosa 03-2024.csv',
 'Dengue IgM- 03-2024.csv',
 'Hemograma 03-2024.csv',
 'Dengue 03-2024.csv']

# Dengue cleaning

In [3]:
paths = "norm_data/Dengue IgM- 03-2024.csv", "norm_data/Dengue 03-2024.csv"
df_dengues = pd.concat([pd.read_csv(path) for path in paths])

print(df_dengues.shape)

(4009, 5)


In [7]:
# Step 1: Sort the DataFrame
df_dengues.sort_values(by=['id_subject', 'resultado', 'fecha_muestra', 'analisis'], ascending = [False, False, True, True], inplace = True)

# Step 2: Keeping the first positive result (if any) of the first date. 
mask = ~df_dengues["id_subject"].duplicated(keep='first')
df = df_dengues[mask]

In [8]:
df

,id_subject,fecha_nacimiento,fecha_muestra,analisis,resultado
399,fffe7e0e6d9720dd75871effd9415640,2000-01-31 00:00:00,2024-03-07,PCR PARA DENGUE.,Positivo
2250,fffb7d7b9f956eaa701a904e81aa2096,1997-10-15,2024-03-27,PCR PARA DENGUE.,Negativo
1547,ffe8fd781009a13194976c05f28f66ad,2007-11-14 00:00:00,2024-03-20,PCR PARA DENGUE.,Positivo
118,ffcb4f9852bc1f6728a7b95235fb3b82,1978-12-21 00:00:00,2024-03-03,PCR PARA DENGUE.,Negativo
1884,ffa85c2cbd6a4a7c2d7d6a5d52a85964,2003-06-05,2024-03-24,PCR PARA DENGUE.,Positivo-DEN1
...,...,...,...,...,...
2257,006da0f996c5b6b4bf121a8263bc3ee5,1980-09-28,2024-03-27,PCR PARA DENGUE.,Positivo-DEN1
1578,003b2732d132c0c8cbdabfc5488b83fd,2015-10-01 00:00:00,2024-03-21,PCR PARA DENGUE.,Positivo
1113,002f9a1a1dcdd9650c3fd7829d167ccb,2006-11-01 00:00:00,2024-03-17,PCR PARA DENGUE.,Negativo
927,00201509ff6946619043c1e8f92db9d8,1980-12-18 00:00:00,2024-03-14,PCR PARA DENGUE.,Positivo-DEN1


# Join data

### Glucosa

In [5]:
labs = ["norm_data/Glucosa 03-2024.csv", 
        "norm_data/Plaquetas 03-2024.csv",
        "norm_data/Hepatograma 03-2024.csv",
        "norm_data/Hemograma 03-2024.csv"]

In [6]:
for path in labs:
    print(f"{path} started with {df.shape}")
    lab = pd.read_csv(path)
    print(path, lab.shape)
    df = pd.merge(df, lab, on = "id_subject", how = "left", suffixes= ["", "_new"])
    print(f"{path} after merge {df.shape}")
    df["delta_dias"] = (pd.to_datetime(df.fecha_muestra) - pd.to_datetime(df.fecha_muestra_new.fillna(df.fecha_muestra))).dt.days
    
    df.loc[~df.delta_dias.between(-1,7), lab.columns[2:]] = np.nan
    print(f"{path} after delta dias {df.shape}")

    df["delta_dias_neg"] = df.delta_dias > 0
    index_cols = ['id_subject',"delta_dias_neg", 'delta_dias']
    df = df.sort_values(by=index_cols, ascending = [True, False, True])
    mask = ~df["id_subject"].duplicated(keep='first')
    df = df[mask]
    df.drop(columns = ["fecha_muestra_new", "delta_dias","delta_dias_neg"], inplace = True)
    print(f"{path} ended with {df.shape}\n")

norm_data/Glucosa 03-2024.csv started with (2818, 5)
norm_data/Glucosa 03-2024.csv (6076, 3)
norm_data/Glucosa 03-2024.csv after merge (3157, 7)
norm_data/Glucosa 03-2024.csv after delta dias (3157, 8)
norm_data/Glucosa 03-2024.csv ended with (2818, 6)

norm_data/Plaquetas 03-2024.csv started with (2818, 6)
norm_data/Plaquetas 03-2024.csv (5665, 5)
norm_data/Plaquetas 03-2024.csv after merge (3656, 10)
norm_data/Plaquetas 03-2024.csv after delta dias (3656, 11)
norm_data/Plaquetas 03-2024.csv ended with (2818, 9)

norm_data/Hepatograma 03-2024.csv started with (2818, 9)
norm_data/Hepatograma 03-2024.csv (6858, 10)
norm_data/Hepatograma 03-2024.csv after merge (3537, 18)
norm_data/Hepatograma 03-2024.csv after delta dias (3537, 19)
norm_data/Hepatograma 03-2024.csv ended with (2818, 17)

norm_data/Hemograma 03-2024.csv started with (2818, 17)
norm_data/Hemograma 03-2024.csv (9121, 23)
norm_data/Hemograma 03-2024.csv after merge (3843, 39)
norm_data/Hemograma 03-2024.csv after delta dias

In [312]:
df.isna().sum() / len(df) * 100

id_subject                                   0.000000
fecha_nacimiento                             0.000000
fecha_muestra                                0.000000
analisis                                     0.000000
resultado                                    0.000000
glucosa                                     73.314407
observaciones_de_plaquetas                  93.718950
recuento_de_plaquetas                       51.135557
no_plaq                                     51.100071
albumina                                    56.422995
bilirrubina_directa                         56.032647
bilirrubina_total                           56.032647
colesterol                                  56.032647
fosfatasa_alcalina_(fal)                    56.032647
proteinas_totales                           56.032647
transaminasa_glutamico_oxalacetica_(got)    56.068133
transaminasa_glutamico_piruvica_(gpt)       56.032647
perc_basofilos                              48.154720
perc_eosinofilos            

In [313]:
df["edad"] = ( pd.to_datetime(df.fecha_muestra) -pd.to_datetime(df["fecha_nacimiento"])).dt.days // 365

In [314]:
df["resultado_bin"] = df.resultado.str.contains("Posit").astype(int).map({1:"Positivo", 0:"Negativo/Indet"})

In [315]:
# Drop cases without bioq data
print(df.shape)
df = df.loc[df[['glucosa', 'observaciones_de_plaquetas',
       'recuento_de_plaquetas', 'no_plaq', 'albumina', 'bilirrubina_directa',
       'bilirrubina_total', 'colesterol', 'fosfatasa_alcalina_(fal)',
       'proteinas_totales', 'transaminasa_glutamico_oxalacetica_(got)',
       'transaminasa_glutamico_piruvica_(gpt)', 'perc_basofilos',
       'perc_eosinofilos', 'perc_linfocitos', 'perc_monocitos',
       'perc_neutrofilos_cayados', 'perc_neutrofilos_polisegmentados',
       'basofilos_abs', 'concentracion_de_hb_corp_media', 'eosinofilos_abs',
       'hematocrito', 'hemoglobina', 'hemoglobina_corpuscular_media',
       'linfocitos_abs', 'monocitos_abs', 'neutrofilos_cayados_abs',
       'neutrofilos_polisegmentados_abs', 'observaciones', 'rdw',
       'recuento_de_globulos_blancos', 'recuento_de_globulos_rojos',
       'volumen_corpuscular_medio']].dropna(how = "all").index]

print(df.shape)

(2818, 40)
(1646, 40)


In [316]:
df.drop(columns = ["observaciones_de_plaquetas", "observaciones"], inplace=True)

In [317]:
df.to_csv("clean_data.csv", index = None)